In [1]:
import cv2
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
labels=['Cancer', 'Non Cancer']
img_path='Skin Data/'

In [3]:
img_list=[]
label_list=[]

for label in labels:
    for img_file in os.listdir(img_path+label):
        img_list.append(img_path+label+'/'+img_file)
        label_list.append(label)

In [4]:
df = pd.DataFrame({'img': img_list, 'label': label_list})

In [5]:
df.sample(5)

,img,label
130,Skin Data/Non Cancer/614.JPG,Non Cancer
73,Skin Data/Cancer/2301-1.JPG,Cancer
202,Skin Data/Non Cancer/1111.JPG,Non Cancer
211,Skin Data/Non Cancer/1248-1.JPG,Non Cancer
199,Skin Data/Non Cancer/1065.jpg,Non Cancer


In [6]:
d={'Non Cancer': 0, 'Cancer': 1}
df['encode_label']=df['label'].map(d)

In [7]:
x = []

for img in df['img']:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (170, 170))
    img = img / 255.0 #normalize
    x.append(img)

In [8]:
x = np.array(x)

In [9]:
y=df['encode_label']

In [10]:
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization, Reshape

2024-05-15 10:21:21.775290: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=.20, random_state=42)

In [12]:
# CNN = Convolutional Neural Networks

In [23]:
model=Sequential()
model.add(Input(shape=(170,170,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(2, activation='softmax')) # 10 fakli cevap classification 0-9 a kadar olan rakamlar

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Train the model
history=model.fit( x_train, y_train,validation_data=(x_test,y_test), epochs=15, verbose=1)


Epoch 1/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 660ms/step - accuracy: 0.5291 - loss: 9.7854 - val_accuracy: 0.7414 - val_loss: 2.5156
Epoch 2/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 610ms/step - accuracy: 0.6474 - loss: 2.9485 - val_accuracy: 0.2586 - val_loss: 0.7912
Epoch 3/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 623ms/step - accuracy: 0.6237 - loss: 0.6547 - val_accuracy: 0.7586 - val_loss: 0.5047
Epoch 4/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 669ms/step - accuracy: 0.7573 - loss: 0.5762 - val_accuracy: 0.7931 - val_loss: 0.4346
Epoch 5/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 650ms/step - accuracy: 0.7664 - loss: 0.4830 - val_accuracy: 0.7414 - val_loss: 0.6113
Epoch 6/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 619ms/step - accuracy: 0.7919 - loss: 0.4656 - val_accuracy: 0.8448 - val_loss: 0.3715
Epoch 7/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 643ms/step - accuracy: 0.8623 - loss: 0.3305 - val_accuracy: 0.8276 - val_loss: 0.4111
Epoch 8/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 617ms/step - accuracy: 0.8871 - loss: 0.3118 - val_accuracy: 0.8103 - val_loss:

In [20]:
model.save('cnn_model.h5')

In [ ]:
# VGGNET, ResNet50, Inceptionv3, Xception, MobileNetv2 Transfer Learning

In [27]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization, Reshape

from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [30]:
data_dir='Skin Data'
img_width,img_heigth=224,224

train_datagen=ImageDataGenerator(rescale=1/255, validation_split=.20)
train_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_width,img_heigth),
                                                      class_mode='binary', subset='training')

 
test_datagen=ImageDataGenerator(rescale=1/255)
test_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_width,img_heigth),
                                                     class_mode='binary', subset='validation')

 
base_model=VGG16(weights='imagenet', input_shape=(img_width,img_heigth,3),include_top=False)

model=Sequential()

model.add(base_model)
for layer in base_model.layers:
    layer.trainable=False

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(train_datagenerator,epochs=10,validation_data=test_datagenerator)

Found 232 images belonging to 2 classes.
Found 56 images belonging to 2 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/10


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 43s 5s/step - accuracy: 0.5209 - loss: 5.4695 - val_accuracy: 0.7143 - val_loss: 1.6115
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 41s 5s/step - accuracy: 0.6928 - loss: 2.1946 - val_accuracy: 0.3036 - val_loss: 1.9652
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 38s 5s/step - accuracy: 0.5871 - loss: 1.1498 - val_accuracy: 0.7679 - val_loss: 0.5415
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 38s 5s/step - accuracy: 0.8169 - loss: 0.4627 - val_accuracy: 0.7679 - val_loss: 0.5914
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 37s 5s/step - accuracy: 0.8383 - loss: 0.3790 - val_accuracy: 0.7857 - val_loss: 0.4250
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 37s 5s/step - accuracy: 0.9351 - loss: 0.1650 - val_accuracy: 0.8393 - val_loss: 0.3612
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 34s 4s/step - accuracy: 0.9531 - loss: 0.1619 - val_accuracy: 0.8393 - val_loss: 0.3391
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step - accuracy: 0.9621 - loss: 0.1155 - val_accuracy: 0.8393 - val_loss: 0.3643
Epoch 9/10
8/8 ━━━━

In [31]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1024)           │    25,691,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,791,173 (350.16 MB)

 Trainable params: 25,692,161 (98.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 51,384,324 (196.02 MB)